In [78]:
import random
import pandas as pd

### Find find extrema(min, max) of a function:
\begin{equation*}
\frac{x - 3}{(x + 5)} - (x + 3)(x - 5), x  \in [15, 45], x\not=5
\end{equation*}

In [79]:
def fun(x):
    assert x != 5
    return ((x - 3) / (x + 5)) - ((x + 3) * (x - 5))

def negative_fun(y):
    return -fun(y)

values = sorted([(fun(i), i) for i in range(15, 46)], 
                    key=lambda x: x[0], 
                    reverse=True)
actual_max, _ = values[0]
actual_min, _ = values[-1]
print(f'actual max = { actual_max }')
print(f'actual min = { actual_min }')

actual max = -179.4
actual min = -1919.16


### Converting to gray code 

In [80]:
def binary_to_gray(n):
    n = int(n, 2)
    n ^= (n >> 1)
    return bin(n)[2:]

def gray_to_binary(n):
    n = int(n, 2) 
    mask = n
    while mask != 0:
        mask >>= 1
        n ^= mask
    return bin(n)[2:]

def int_to_gray(n):
    return binary_to_gray(bin(n))


def gray_to_int(n):
    return int(gray_to_binary(n), base=2)

### Chromosome representation

In [81]:
def chromosome_bits(n, bits_len=None):
    # return bin(n)
    if bits_len:
        return fill_zeros(int_to_gray(n), bits_len)
    return int_to_gray(n)
    
def phenotype(n):
    # return int(n, base=2)
    return gray_to_int(n)

assert chromosome_bits(10) == '1111'
assert phenotype('1111') == 10
assert phenotype(chromosome_bits(1234)) == 1234

### Util operations

In [82]:
def invert_bit(bits, i):
    x = list(bits)
    x[i] = int(not int(x[i]))
    return ''.join(map(str, x))

def fill_zeros(bits, length):
    b = list(bits)
    while len(b) != length:
        b.insert(0, 0)
    return ''.join(map(str, b))

def make_pairs(_population):
    l = list(range(0, len(_population)))
    random.shuffle(l)
    pairs = [(l[i], l[i + 1]) for i in range(0, len(l) - 1)]
    pairs.append((l[0], l[-1]))
    return pairs

### Genetic operators

* Crossing
    * rift = 3
        - **000**000 >> 000 | 111 = 000111
        - **111**111 >> 111 | 000 = 111000
        
    * rift = 1
        - **1**111 >> 1 | 000 = 1000
        - **0**000 >> 0 | 111 = 0111

In [83]:
def crossing(chromosome1, chromosome2, rift=None, probability=0.8):
    def cross():
        assert len(chromosome1) == len(chromosome2)
        if rift is None:
            pos = random.randint(0, len(chromosome1))
        else:
            pos = rift
        assert 0 <= pos <= len(chromosome1)
        return (''.join([chromosome1[:pos], chromosome2[pos:]]),
                ''.join([chromosome2[:pos], chromosome1[pos:]]))
    
    assert (0 <= probability <= 1)
    
    if random.random() < probability:
        return cross()
    return chromosome1, chromosome2


assert crossing('000000', '111111', 3, 1) == ('000111', '111000')
assert crossing('1111', '0000', 1, 1) == ('1000', '0111')

def cross_population(_population, probability=0.8):
    pairs = make_pairs(_population)
    return [crossing(_population[i1], _population[i2], 
                     probability=probability)[0] 
            for i1, i2 in pairs]

* Mutation(bit_position=3)
    - 000**0**00 >> 000**1**00

In [84]:
def mutation(chromosome, 
             probability=0.2, 
             bit_position=None, 
             allow_range=range(15, 46)):
    def mutate():
        if bit_position is None:
            pos = random.randint(0, len(chromosome) - 1)
        else:
            pos = bit_position
        return invert_bit(chromosome, pos)

    assert (0 <= probability <= 1)
    
    if random.random() < probability:
        result = mutate()
        while phenotype(result) not in allow_range:
            result = mutate()
        return result
    return chromosome

assert mutation('1111', bit_position=1, probability=0) == '1111'
assert mutation(chromosome_bits(10), bit_position=0, probability=1, allow_range=range(0, 20)) == '0111'
assert mutation(chromosome_bits(10), bit_position=1, probability=1, allow_range=range(0, 20)) == '1011'
assert mutation(chromosome_bits(10), bit_position=2, probability=1, allow_range=range(0, 20)) == '1101'
assert mutation(chromosome_bits(10), bit_position=3, probability=1, allow_range=range(0, 20)) == '1110'

def mutate_population(_population, probability=0.2):
    return [mutation(ch, probability) for ch in population] 

### Genetic selection

In [85]:
def population_data(_population):
    df = pd.DataFrame(data=[ch_i for ch_i in _population], columns=['chromosome'])
    df['phenotype'] = df.apply(lambda row: phenotype(row['chromosome']), axis=1) 
    df['fun_value'] = df.apply(lambda row: fun(row['phenotype']), axis=1)
    df['sel_probability'] = df['fun_value'] / df['fun_value'].sum()
    return df

def roulette(chromosomes, survive_probabilities):
    n = len(chromosomes)
    assert n == len(chromosomes) == len(survive_probabilities)
    l = [i for i, _ in sorted(zip(chromosomes, survive_probabilities), 
                              key=lambda x: x[1])]
    return l[:n]

def select_new_population(df):
    return roulette(chromosomes=df['chromosome'].values,
                    survive_probabilities=df['sel_probability'].values)

### Initial values

In [86]:
# population contains N chromosomes 
# each chromosome consist of L bits
N = 4
L = len(chromosome_bits(45))

population = [
    chromosome_bits(15, L),
    chromosome_bits(32, L),
    chromosome_bits(27, L),
    chromosome_bits(15, L),
]

print('Initial population:')
print(population_data(population))
assert len(population) == N
assert L == 6

Initial population:
  chromosome  phenotype   fun_value  sel_probability
0     001000         15 -179.400000         0.091425
1     110000         32 -944.216216         0.481187
2     010110         27 -659.250000         0.335964
3     001000         15 -179.400000         0.091425


### Genetic algorithm

In [87]:
count = 0
for i in range(100):
    temp = population.copy()
    population = select_new_population(population_data(population))
    print(f'{population_data(population)}\n')
    population = cross_population(population, 
                                  probability=0.7)
    population = mutate_population(population, 
                                   probability=0.2)
    count += 1

  chromosome  phenotype   fun_value  sel_probability
0     001000         15 -179.400000         0.091425
1     001000         15 -179.400000         0.091425
2     010110         27 -659.250000         0.335964
3     110000         32 -944.216216         0.481187

  chromosome  phenotype   fun_value  sel_probability
0     000000          0   14.400000        -0.017019
1     000110          4    7.111111        -0.008404
2     011000         16 -208.380952         0.246278
3     010110         27 -659.250000         0.779145

  chromosome  phenotype  fun_value  sel_probability
0     000110          4   7.111111         0.165289
1     000110          4   7.111111         0.165289
2     000000          0  14.400000         0.334711
3     000000          0  14.400000         0.334711

  chromosome  phenotype  fun_value  sel_probability
0     000110          4   7.111111         0.148423
1     000010          3  12.000000         0.250464
2     000000          0  14.400000         0.300557

In [88]:
print(f'Took {count} iterations')
print(population_data(population))

Took 100 iterations
  chromosome  phenotype    fun_value  sel_probability
0     011000         16  -208.380952         0.087839
1     011000         16  -208.380952         0.087839
2     111110         43 -1747.166667         0.736483
3     011000         16  -208.380952         0.087839
